## Load the data
## Clean the data
## Modelling

In [42]:
import pandas as pd

In [43]:
data = pd.read_csv('./data.csv')

In [44]:
X,y = data.drop('target',axis=1),data['target']

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [47]:
import torch
import torch.nn as nn

In [48]:
import numpy as np

In [49]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [50]:
X_train.shape

torch.Size([227, 13])

In [51]:
X_test.shape

torch.Size([76, 13])

In [52]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

#### Modelling

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,128)
        self.fc4 = nn.Linear(128,256)
        self.fc5 = nn.Linear(256,512)
        self.fc6 = nn.Linear(512,256)
        self.fc7 = nn.Linear(256,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
model = Test_Model().to(device)

In [17]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)

In [18]:
criterion = nn.L1Loss()

In [19]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [20]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [21]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [22]:
import wandb

In [23]:
from tqdm import tqdm

In [24]:
EPOCHS = 100

In [25]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|          | 0/100 [00:00<?, ?it/s]/home/indika/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/indika/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([76])) that is different to the input size (torch.Size([76, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
100%|██████████| 100/100 [00:09<00:00, 10.56it/s]


loss,0.39971
val_loss,0.47816
accuracy,0.449
val_accuracy,0.474
_runtime,16
_timestamp,1621445516
_step,99


loss,█▆▅▄▄▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▁
val_loss,▆▄▂▂▃█▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂
accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [26]:
preds_new = []

In [40]:
torch.round(preds)[:12]

tensor([0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [41]:
y_train[:12]

tensor([0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.], device='cuda:0')